<a href="https://colab.research.google.com/github/martinkenas/Road-Segmentation-for-type-wetness-and-roughness/blob/main/model_training_scripts/Kelio_pavirsius_YOLO8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title PILNAS PAVIRSIUS 1. Nustatymas ir diegimas

# Įdiegiamos reikalingos bibliotekos
!pip install ultralytics

# Importuojamos būtinos bibliotekos
import os
import yaml
import shutil
import random
from IPython.display import display, Image
from ultralytics import YOLO
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image as PILImage
import cv2
from google.colab import drive
from datetime import datetime
import torch
from sklearn.model_selection import train_test_split

# Prijungiamas Google Drive
drive.mount('/content/drive')

print("Nustatymas baigtas!")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 116.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 97.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [ ]:
#@title 2. Projekto struktūros konfigūravimas

import os
import yaml
import random
import shutil
from datetime import datetime
from PIL import Image as PILImage
from sklearn.model_selection import train_test_split

base_dir = '/content/drive/MyDrive/Dangos Nustatymas'
print(f"Pagrindinis katalogas: {base_dir}")

# Pradiniai katalogai su duomenimis - naudojamas labels2 katalogas
images_dir = os.path.join(base_dir, 'images')
labels_dirs = {
    'asfaltas': os.path.join(base_dir, 'labels2', 'asfaltas'),
    'zvyras': os.path.join(base_dir, 'labels2', 'zvyras'),
    'grindinys': os.path.join(base_dir, 'labels2', 'grindinys')
}

# Sukuriamas laiko žymės formatas unikaliam projekto paleidimui
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
project_dir = os.path.join(base_dir, f'road_seg_{timestamp}')
os.makedirs(project_dir, exist_ok=True)

# Rezultatų katalogai
results_dir = os.path.join(project_dir, 'results')
predictions_dir = os.path.join(project_dir, 'predictions')
exports_dir = os.path.join(project_dir, 'exports')
os.makedirs(results_dir, exist_ok=True)
os.makedirs(predictions_dir, exist_ok=True)
os.makedirs(exports_dir, exist_ok=True)

# Sukuriamas bendras etikečių katalogas, kurio tikisi YOLO
yolo_labels_dir = os.path.join(base_dir, 'labels')
os.makedirs(yolo_labels_dir, exist_ok=True)
print(f"Sukurtas YOLO etikečių katalogas: {yolo_labels_dir}")


Pagrindinis katalogas: /content/drive/MyDrive/Dangos Nustatymas
Sukurtas YOLO etikečių katalogas: /content/drive/MyDrive/Dangos Nustatymas/labels


In [ ]:
#@title 3. Duomenų rinkinio analizė ir kelio dangos tipų sekimas sverti

def analyze_dataset():
    """Analizuojamas duomenų rinkinys ir sukuriami padalinimai"""
    # Gaunami visi pažymėti failai iš kiekvieno katalogo
    all_files = []

    # Sekamas bendras failų skaičius kiekviename dangos tipe
    type_counts = {}

    for pavement_type, label_dir in labels_dirs.items():
        files = []

        # Gaunami visi txt failai, išskyrus classes.txt
        for file in os.listdir(label_dir):
            if file.endswith('.txt') and file != 'classes.txt':
                base_name = os.path.splitext(file)[0]
                # Tikrinama ar egzistuoja atitinkamas vaizdas
                img_path = None
                for ext in ['.jpg', '.jpeg', '.png']:
                    if os.path.exists(os.path.join(images_dir, base_name + ext)):
                        img_path = os.path.join(images_dir, base_name + ext)
                        break

                if img_path:
                    files.append({
                        'base_name': base_name,
                        'img_path': img_path,
                        'label_path': os.path.join(label_dir, file),
                        'pavement_type': pavement_type
                    })

        all_files.extend(files)
        type_counts[pavement_type] = len(files)

    # Spausdinami statistiniai duomenys
    print("\nDuomenų rinkinio statistika:")
    total_files = sum(type_counts.values())
    for pavement_type, count in type_counts.items():
        print(f"  {pavement_type}: {count} failai ({count/total_files*100:.1f}%)")
    print(f"  Viso: {total_files} failai")

    # Sukuriami padalinimai (70% treniravimui, 20% validavimui, 10% testavimui)
    random.seed(42)  # Atkūrimui
    random.shuffle(all_files)

    train_size = int(0.7 * len(all_files))
    val_size = int(0.2 * len(all_files))

    train_files = all_files[:train_size]
    val_files = all_files[train_size:train_size+val_size]
    test_files = all_files[train_size+val_size:]

    splits = {'train': train_files, 'val': val_files, 'test': test_files}

    # Spausdinami padalinimų statistika
    print("\nPadalinimų statistika (naudojamas svertinis duomenų krovimas, be rankinio balansavimo):")
    for split_name, files in splits.items():
        print(f"  {split_name}: {len(files)} failai")

        # Skaičiuojama pagal dangos tipą
        split_counts = {}
        for file in files:
            pt = file['pavement_type']
            split_counts[pt] = split_counts.get(pt, 0) + 1

        for pt, count in split_counts.items():
            print(f"    - {pt}: {count} failai ({count/len(files)*100:.1f}%)")

    return splits, type_counts

# Analizuojamas duomenų rinkinys ir sukuriami padalinimai
splits, type_counts = analyze_dataset()



Duomenų rinkinio statistika:
  asfaltas: 663 failai (49.6%)
  zvyras: 331 failai (24.8%)
  grindinys: 343 failai (25.7%)
  Viso: 1337 failai

Padalinimų statistika (naudojamas svertinis duomenų krovimas, be rankinio balansavimo):
  train: 935 failai
    - grindinys: 239 failai (25.6%)
    - asfaltas: 457 failai (48.9%)
    - zvyras: 239 failai (25.6%)
  val: 267 failai
    - asfaltas: 140 failai (52.4%)
    - zvyras: 63 failai (23.6%)
    - grindinys: 64 failai (24.0%)
  test: 135 failai
    - zvyras: 29 failai (21.5%)
    - asfaltas: 66 failai (48.9%)
    - grindinys: 40 failai (29.6%)


In [ ]:
#@title 4. Duomenų rinkinio konfigūracijos failų sukūrimas ir etikečių normalizavimas

def copy_and_normalize_labels():
    """Kopijuojamos etiketės iš pradinių vietų į standartinę YOLO struktūrą su 0 klase"""
    # Išvalomi visi esami failai etikečių kataloge, kad būtų išvengta konfliktų
    for file in os.listdir(yolo_labels_dir):
        if file.endswith('.txt'):
            os.remove(os.path.join(yolo_labels_dir, file))

    # Kopijuojami visi etikečių failai į konsoliduotą katalogą, konvertuojant visas klases į 0 klasę (kelias)
    label_count = 0
    for split_name, files in splits.items():
        for file_data in files:
            # Pradinis etiketės kelias
            original_label_path = file_data['label_path']
            # Pagrindinis vaizdo/etiketės pavadinimas
            base_name = file_data['base_name']
            # Tikslo kelias YOLO struktūroje
            target_label_path = os.path.join(yolo_labels_dir, f"{base_name}.txt")

            # Skaitomas etiketės turinys ir užtikrinama, kad visos klasės nustatytos į 0
            try:
                with open(original_label_path, 'r') as src_file:
                    label_lines = src_file.readlines()

                # Rašomos normalizuotos etiketės (visos klasės konvertuotos į 0 klasę)
                with open(target_label_path, 'w') as dst_file:
                    for line in label_lines:
                        parts = line.strip().split()
                        if len(parts) > 0:
                            # Nustatoma klasė į 0 (kelias), išsaugomos visos kitos reikšmės
                            parts[0] = '0'
                            dst_file.write(' '.join(parts) + '\n')

                label_count += 1
            except Exception as e:
                print(f"Klaida apdorojant etiketę failui {base_name}: {e}")

    print(f"Normalizuota ir nukopijuota {label_count} etiketės failai į {yolo_labels_dir}")
    return label_count

def create_dataset_config():
    """Sukuriami duomenų rinkinio konfigūracijos failai"""
    # Pirmiausia kopijuojamos etiketės į YOLO struktūrą, normalizuojant klasių ID
    label_count = copy_and_normalize_labels()

    if label_count == 0:
        print("ĮSPĖJIMAS: Nebuvo nukopijuota nė viena etiketė. Patikrinkite etikečių katalogus.")

    # Sukuriama duomenų rinkinio katalogų struktūra, kurios tikisi YOLOv8
    for split_name in splits.keys():
        split_img_dir = os.path.join(base_dir, 'images', split_name)
        os.makedirs(split_img_dir, exist_ok=True)

    # Sukuriami treniravimo/validavimo/testavimo tekstiniai failai, išvardijantys kelius
    for split_name, files in splits.items():
        split_file = os.path.join(project_dir, f'{split_name}.txt')
        with open(split_file, 'w') as f:
            for file_data in files:
                f.write(f"{file_data['img_path']}\n")

    # Sukuriamas duomenų rinkinio YAML failas su aiškiai nurodytu etikečių katalogu
    dataset_yaml = os.path.join(project_dir, 'dataset.yaml')
    dataset_config = {
        'path': base_dir,
        'train': os.path.join(project_dir, 'train.txt'),
        'val': os.path.join(project_dir, 'val.txt'),
        'test': os.path.join(project_dir, 'test.txt'),
        'names': {0: 'road'},  # Viena klasė: kelias
        'labels': yolo_labels_dir  # Aiškiai nurodomas etikečių katalogas
    }

    with open(dataset_yaml, 'w') as f:
        yaml.dump(dataset_config, f)

    return dataset_yaml

# Sukuriami konfigūracijos failai
dataset_yaml = create_dataset_config()
print(f"\nDuomenų rinkinio konfigūracija sukurta: {dataset_yaml}")

Normalizuota ir nukopijuota 1337 etiketės failai į /content/drive/MyDrive/Dangos Nustatymas/labels

Duomenų rinkinio konfigūracija sukurta: /content/drive/MyDrive/Dangos Nustatymas/road_seg_20250504_181219/dataset.yaml


In [ ]:
#@title 5. Svertinio duomenų kroviklio įgyvendinimas YOLO pagal kelio paviršiaus tipus

from ultralytics.data.dataset import YOLODataset
import numpy as np

class YOLOSurfaceWeightedDataset(YOLODataset):
    def __init__(self, *args, surface_type_map=None, surface_weights=None, **kwargs):
        """
        Inicializuojamas svertinis duomenų rinkinys, paremtas kelio paviršių tipais.

        Args:
            surface_type_map: Žodynas, susiejantis failų pagrindinius pavadinimus su paviršiaus tipais
            surface_weights: Žodynas, susiejantis paviršiaus tipus su svoriais
        """
        super(YOLOSurfaceWeightedDataset, self).__init__(*args, **kwargs)

        self.train_mode = "train" in self.prefix

        # Svoriai taikomi tik treniravimo režime
        if self.train_mode and surface_type_map and surface_weights:
            self.surface_type_map = surface_type_map
            self.surface_weights = surface_weights

            # Apskaičiuojami vaizdų svoriai pagal paviršiaus tipą
            self.weights = self.calculate_weights()
            self.probabilities = self.calculate_probabilities()

            print(f"Paviršiaus tipų svoriai: {self.surface_weights}")

    def calculate_weights(self):
        """
        Apskaičiuojamas svoris kiekvienam vaizdui pagal jo paviršiaus tipą.
        """
        weights = []
        for img_path in self.im_files:
            # Išgaunamas pagrindinis pavadinimas iš vaizdo kelio
            base_name = os.path.splitext(os.path.basename(img_path))[0]

            # Gaunamas paviršiaus tipas ir priskiriamas svoris
            surface_type = self.surface_type_map.get(base_name, 'unknown')
            weight = self.surface_weights.get(surface_type, 1.0)
            weights.append(weight)

        return weights

    def calculate_probabilities(self):
        """
        Apskaičiuojamos atrankos tikimybės pagal svorius.
        """
        total_weight = sum(self.weights)
        probabilities = [w / total_weight for w in self.weights]
        return probabilities

    def __getitem__(self, index):
        """
        Grąžinama transformuota etiketės informacija pagal atrenkamą indeksą.
        """
        # Nenaudojama svertinė atranka validavimui/testavimui
        if not self.train_mode or not hasattr(self, 'probabilities'):
            return self.transforms(self.get_image_and_label(index))
        else:
            # Naudojama svertinė atranka treniravimui
            index = np.random.choice(len(self.im_files), p=self.probabilities)
            return self.transforms(self.get_image_and_label(index))


def create_surface_type_map():
    """Sukuriamas susiejimas tarp failų pagrindinių pavadinimų ir paviršiaus tipų"""
    surface_map = {}
    for split_name, files in splits.items():
        for file_data in files:
            base_name = file_data['base_name']
            surface_type = file_data['pavement_type']
            surface_map[base_name] = surface_type
    return surface_map

def calculate_surface_weights(type_counts):
    """Apskaičiuojami svoriai kiekvienam paviršiaus tipui pagal atvirkštinį dažnį"""
    total = sum(type_counts.values())
    weights = {surface: total / count for surface, count in type_counts.items()}
    return weights

# Sukuriamas paviršiaus tipų susiejimas ir svoriai
surface_type_map = create_surface_type_map()
surface_weights = calculate_surface_weights(type_counts)

# Pritaikomas pasirinktinis duomenų rinkinio klasė per monkey patching
from functools import partial
import ultralytics.data.build as build

# Sukuriama dalinė funkcija su iš anksto apskaičiuotais žemėlapiais
CustomDataset = partial(
    YOLOSurfaceWeightedDataset,
    surface_type_map=surface_type_map,
    surface_weights=surface_weights
)

# Monkey-patching YOLODataset klasei
build.YOLODataset = CustomDataset

print("Svertinis duomenų kroviklio įgyvendinimas baigtas!")
print(f"Paviršiaus tipų svoriai: {surface_weights}")

Svertinis duomenų kroviklio įgyvendinimas baigtas!
Paviršiaus tipų svoriai: {'asfaltas': 2.0165912518853695, 'zvyras': 4.039274924471299, 'grindinys': 3.8979591836734695}


In [ ]:
#@title 6. YOLOv11 Modelio Apmokymas su Integruotu Validavimu/Eksportavimu

def train_and_export_model():
    """Apmokyti YOLOv8n-seg modelį su integruotu validavimu, testavimu ir eksportavimu"""
    # Importuoti YOLO
    from ultralytics import YOLO
    import torch
    import glob
    import time
    import os
    import logging

    # Set up basic logging for better error tracking
    logging.basicConfig(level=logging.INFO,
                      format='%(asctime)s - %(levelname)s - %(message)s')
    logger = logging.getLogger('train_export')

    # Try to use the fastest available segmentation model
    try:
        # First try YOLOv11n-seg if available (newest/fastest)
        logger.info("Trying to load YOLOv11n-seg model...")
        model = YOLO('yolov8n-seg.pt')
        logger.info("Successfully loaded YOLOv11n-seg model!")
    except Exception as e:
        logger.warning(f"YOLOv11n-seg not available: {str(e)}")
        try:
            # Try YOLOv10n-seg if available (second fastest)
            logger.info("Trying to load YOLOv10n-seg model...")
            model = YOLO('yolov10n-seg.pt')
            logger.info("Successfully loaded YOLOv10n-seg model!")
        except:
            # Fall back to YOLOv8n-seg (reliable option)
            logger.info("Falling back to YOLOv8n-seg model")
            model = YOLO('yolov8n-seg.pt')

    # Log which model we're using
    logger.info(f"Using model: {model.type} - {model.model.names}")

    # Apibrėžti augmentacijos konfigūraciją
    augmentation_config = {
        'hsv_h': 0.15,     # Atspalvio variacija spalvų pakeitimui
        'hsv_s': 0.5,      # Stipri sodrumo variacija snieginėms sąlygoms
        'hsv_v': 0.5,      # Reikšmės (šviesumo) variacija
        'degrees': 20.0,   # Pasukimas - skirtingiems kameros kampams
        'translate': 0.1,  # Poslinkis
        'scale': 0.2,      # Mastelio variacija
        'fliplr': 0.5,     # Horizontalus apvertimas skirtingoms eismo pusėms
        'mosaic': 0.6,     # Mozaikos augmentacija
        'mixup': 0.15,     # Mixup augmentacija
    }

    # Apmokymo konfigūracija
    epochs = 130           # Epochų skaičius
    imgsz = 1024           # Vaizdo dydis
    batch_size = 4         # Fiksuotas batch dydis 4080x3060 vaizdams

    logger.info("\nPradedamas apmokymas su paviršiaus tipo svoriais...")
    print("\nPradedamas apmokymas su paviršiaus tipo svoriais...")
    print("Tai užtruks nemažai laiko.")

    # Patikrinti, ar egzistuoja projekto direktorija
    if not os.path.exists(project_dir):
        os.makedirs(project_dir, exist_ok=True)
        logger.info(f"Sukurta projekto direktorija: {project_dir}")

    # Įsitikinti, kad exports_dir egzistuoja
    if not os.path.exists(exports_dir):
        os.makedirs(exports_dir, exist_ok=True)
        logger.info(f"Sukurta eksportavimo direktorija: {exports_dir}")

    # Įsitikinti, kad results_dir egzistuoja
    if not os.path.exists(results_dir):
        os.makedirs(results_dir, exist_ok=True)
        logger.info(f"Sukurta rezultatų direktorija: {results_dir}")

    # Pradėti apmokymą su modeliu ir svertiniu duomenų krovikliu
    try:
        # Slopinti išsamius pranešimus apmokymo metu
        import builtins
        original_print = builtins.print
        builtins.print = lambda *args, **kwargs: None

        # Pradėti apmokymą
        results = model.train(
            data=dataset_yaml,
            epochs=epochs,
            imgsz=imgsz,
            batch=batch_size,
            patience=20,       # Ankstyvo stabdymo kantrybė
            save=True,
            project=project_dir,
            name='model',
            save_period=10,    # Išsaugoti kontrolinius taškus kas 10 epochų
            pretrained=True,
            optimizer='AdamW',  # AdamW optimizatorius
            weight_decay=0.01,  # Svorių mažėjimas
            lr0=0.0005,         # Pradinė mokymosi sparta
            lrf=0.001,          # Galutinės mokymosi spartos faktorius
            augment=True,       # Įjungti augmentaciją
            cache='disk',       # Naudoti disko podėlį deterministiniams rezultatams
            device=0 if torch.cuda.is_available() else 'cpu', # Įrenginys
            plots=True,         # Generuoti grafikus
            single_cls=True,    # Viena klasė (kelias)
            cos_lr=True,        # Išlaikyti kosinuso LR planuotoją
            warmup_epochs=3,    # Pridėti apšilimo periodą
            close_mosaic=15,    # Išjungti mozaiką paskutinėse epochose stabilumui
            **augmentation_config  # Pritaikyti pasirinktus augmentacijos nustatymus
        )

        # Atkurti spausdinimo funkciją
        builtins.print = original_print
        logger.info("Apmokymas baigtas!")
        print("Apmokymas baigtas!")

    except Exception as e:
        # Atkurti spausdinimo funkciją jei įvyksta klaida
        builtins.print = original_print
        logger.error(f"Klaida apmokymo metu: {str(e)}")
        print(f"Klaida apmokymo metu: {str(e)}")
        raise

    # Palaukti momentą, kad failų sistemos operacijos būtų baigtos
    time.sleep(2)

    # Rasti geriausio modelio kelią
    best_model_path = os.path.join(project_dir, 'yolov8n-seg', 'weights', 'best.pt')

    if not os.path.exists(best_model_path):
        # Bandyti rasti geriausio modelio failą
        potential_models = glob.glob(os.path.join(project_dir, 'model', 'weights', '*.pt'))
        if potential_models:
            best_model_path = potential_models[0]
            logger.info(f"Geriausias modelis nerastas numatytame kelyje. Naudojamas rastas modelis: {best_model_path}")
        else:
            logger.error("Nepavyko rasti apmokyto modelio. Patikrinkite išvesties direktorijas.")
            print("Nepavyko rasti apmokyto modelio. Patikrinkite išvesties direktorijas.")
            return None, results, None, None

    logger.info(f"Naudojamas modelis: {best_model_path}")
    print(f"Naudojamas modelis: {best_model_path}")

    # Įkelti geriausią modelį validavimui ir eksportavimui
    best_model = YOLO(best_model_path)

    # Nedelsiant vykdyti validavimą validavimo rinkinyje
    logger.info("\nVykdomas validavimas...")
    print("\nVykdomas validavimas...")
    try:
        val_results = best_model.val(
            data=dataset_yaml,
            split='val',
            imgsz=imgsz,
            batch=batch_size,
            plots=True,
            save_json=True,
            save_txt=True,
            project=results_dir,
            name='validation'
        )

        # Naudoti tinkamą atributo prieigą segmentacijos metrikoms
        if hasattr(val_results, 'seg') and val_results.seg is not None:
            logger.info(f"Validavimo rezultatai: mAP50 = {val_results.seg.map50:.4f}")
            print(f"Validavimo rezultatai: mAP50 = {val_results.seg.map50:.4f}")
            if hasattr(val_results.seg, 'map'):
                logger.info(f"Validavimo rezultatai: mAP50-95 = {val_results.seg.map:.4f}")
                print(f"Validavimo rezultatai: mAP50-95 = {val_results.seg.map:.4f}")
    except Exception as e:
        logger.error(f"Klaida validavimo metu: {str(e)}")
        print(f"Klaida validavimo metu: {str(e)}")
        print("Tęsiamas eksportavimas...")
        val_results = None

    # Apibrėžti išvesties failų pavadinimus ir kelius
    pt_filename = f'model_road_seg_{model_name}.pt'
    onnx_filename = f'model_road_seg_{model_name}.onnx'
    onnx_640_filename = f'model_road_seg_{model_name}_640.onnx'

    # Eksportuoti modelius į skirtingus formatus
    logger.info("\nEksportuojami modeliai...")
    print("\nEksportuojami modeliai...")
    export_success = {'pt': False, 'onnx': False, 'onnx_640': False}

    try:
        # Eksportuoti į .pt formatą (PyTorch)
        pt_save_path = os.path.join(exports_dir, pt_filename)
        best_model.save(pt_save_path)  # Naudoti save .pt formatui
        logger.info(f"PyTorch modelis eksportuotas į: {pt_save_path}")
        print(f"PyTorch modelis eksportuotas į: {pt_save_path}")
        export_success['pt'] = True

        # Eksportuoti į ONNX formatą
        onnx_save_path = os.path.join(exports_dir, onnx_filename)

        logger.info("Eksportuojama į ONNX formatą su išsamiais parametrais...")
        print("Eksportuojama į ONNX formatą su išsamiais parametrais...")

        # Eksportuojame su papildomais paramatrais
        best_model.export(
            format='onnx',
            imgsz=imgsz,        # Use same image size as training
            simplify=True,      # Simplify the model
            opset=12,           # Use ONNX opset 12 for better compatibility
            half=False,         # Don't use half precision to avoid issues
            int8=False,         # Don't use int8 quantization
            dynamic=True,       # Allow dynamic batch size
            verbose=True        # Show detailed export info
        )

        # ONNX models are typically saved in the same directory as the PT model with a .onnx extension
        default_onnx_path = os.path.splitext(best_model_path)[0] + '.onnx'

        # If that's not found, check if it was saved in the current directory
        if not os.path.exists(default_onnx_path):
            for root, dirs, files in os.walk(project_dir):
                for file in files:
                    if file.endswith('.onnx'):
                        default_onnx_path = os.path.join(root, file)
                        break
                if default_onnx_path != os.path.splitext(best_model_path)[0] + '.onnx':
                    break

        if os.path.exists(default_onnx_path):
            import shutil
            # Use copy instead of move to ensure original is kept
            shutil.copy(default_onnx_path, onnx_save_path)
            logger.info(f"ONNX modelis eksportuotas į: {onnx_save_path}")
            print(f"ONNX modelis eksportuotas į: {onnx_save_path}")
            export_success['onnx'] = True

            # Validate the ONNX model to ensure it's working correctly
            try:
                import onnx
                onnx_model = onnx.load(onnx_save_path)
                onnx.checker.check_model(onnx_model)
                logger.info("ONNX modelis sėkmingai patikrintas - teisingas formatas")
                print("ONNX modelis sėkmingai patikrintas - teisingas formatas")
            except Exception as e:
                logger.warning(f"ONNX modelio validavimo įspėjimas: {str(e)}")
                print(f"ONNX modelio validavimo įspėjimas: {str(e)}")
        else:
            logger.error(f"ONNX eksportavimas baigtas, bet failas nerastas: {default_onnx_path}")
            print(f"ONNX eksportavimas baigtas, bet failas nerastas: {default_onnx_path}")
            print("Bandoma alternatyvų eksportavimo metodą...")


            try:
                best_model.export(
                    format='onnx',
                    imgsz=imgsz,
                    simplify=True,
                    opset=12,
                    half=False,
                    dynamic=True,
                    verbose=True,
                    file=onnx_save_path
                )

                if os.path.exists(onnx_save_path):
                    logger.info(f"ONNX modelis eksportuotas į: {onnx_save_path} (alternatyvus metodas)")
                    print(f"ONNX modelis eksportuotas į: {onnx_save_path} (alternatyvus metodas)")
                    export_success['onnx'] = True
            except Exception as e:
                logger.error(f"Alternatyvus ONNX eksportavimas nepavyko: {str(e)}")
                print(f"Alternatyvus ONNX eksportavimas nepavyko: {str(e)}")

        # Papildomas eksportavimas
        onnx_640_save_path = os.path.join(exports_dir, onnx_640_filename)
        logger.info("Eksportuojama į ONNX formatą su 640x640 vaizdo dydžiu...")
        print("Eksportuojama į ONNX formatą su 640x640 vaizdo dydžiu...")

        try:
            # Direct export with 640x640 image size
            best_model.export(
                format='onnx',
                imgsz=640,        # Naudoti 640x640 vaizdo dydi
                simplify=True,
                opset=12,
                half=False,
                int8=False,
                dynamic=True,
                verbose=True,
                file=onnx_640_save_path
            )

            if os.path.exists(onnx_640_save_path):
                logger.info(f"ONNX 640x640 modelis eksportuotas į: {onnx_640_save_path}")
                print(f"ONNX 640x640 modelis eksportuotas į: {onnx_640_save_path}")
                export_success['onnx_640'] = True

                # Modelio validacija
                try:
                    import onnx
                    onnx_model = onnx.load(onnx_640_save_path)
                    onnx.checker.check_model(onnx_model)
                    logger.info("ONNX 640x640 modelis sėkmingai patikrintas - teisingas formatas")
                    print("ONNX 640x640 modelis sėkmingai patikrintas - teisingas formatas")
                except Exception as e:
                    logger.warning(f"ONNX 640x640 modelio validavimo įspėjimas: {str(e)}")
                    print(f"ONNX 640x640 modelio validavimo įspėjimas: {str(e)}")
            else:
                logger.error(f"ONNX 640x640 eksportavimas baigtas, bet failas nerastas")
                print(f"ONNX 640x640 eksportavimas baigtas, bet failas nerastas")
        except Exception as e:
            logger.error(f"ONNX 640x640 eksportavimas nepavyko: {str(e)}")
            print(f"ONNX 640x640 eksportavimas nepavyko: {str(e)}")

    except Exception as e:
        logger.error(f"Klaida modelio eksportavimo metu: {str(e)}")
        print(f"Klaida modelio eksportavimo metu: {str(e)}")
        print(f"Eksportavimo klaidos detalės: {e.__class__.__name__}")
        import traceback
        traceback.print_exc()

    # Generuoti apmokymo santrauką
    print("\n" + "="*50)
    print("APMOKYMO SANTRAUKA")
    print("="*50)

    # Modelio detalės
    model_type = model.type if hasattr(model, 'type') else "YOLOv8n-seg"  # Fallback if attribute missing
    print(f"Modelis: {model_type} su paviršiaus tipo svertiniu duomenų krovikliu")
    print(f"Vaizdo dydis: {imgsz}x{imgsz}")
    print(f"Apmokymo epochos: {epochs}")
    print(f"Partijos dydis: {batch_size}")
    print(f"Viena klasė: kelias (su subalansuotu paviršiaus tipo atrankinimu)")

    # Apmokymo našumas
    if hasattr(results, 'metrics') and results.metrics is not None:
        if hasattr(results.metrics, 'fitness') and results.metrics.fitness is not None:
            print(f"Galutinis tinkamumo balas: {results.metrics.fitness:.4f}")

    # Tinkamai patikrinti segmentacijos metrikas
    if hasattr(results, 'seg'):
        print(f"Segmentacijos našumas:")
        if hasattr(results.seg, 'map50'):
            print(f"  mAP50: {results.seg.map50:.4f}")
        if hasattr(results.seg, 'map'):
            print(f"  mAP50-95: {results.seg.map:.4f}")
        if hasattr(results.seg, 'precision'):
            print(f"  Tikslumas: {results.seg.precision:.4f}")
        if hasattr(results.seg, 'recall'):
            print(f"  Atšaukimas: {results.seg.recall:.4f}")

    # Validavimo našumas
    if val_results is not None and hasattr(val_results, 'seg'):
        print("\nValidavimo našumas:")
        if hasattr(val_results.seg, 'map50'):
            print(f"  Segmentacijos mAP50: {val_results.seg.map50:.4f}")
        if hasattr(val_results.seg, 'map'):
            print(f"  Segmentacijos mAP50-95: {val_results.seg.map:.4f}")
        if hasattr(val_results.seg, 'precision'):
            print(f"  Segmentacijos tikslumas: {val_results.seg.precision:.4f}")
        if hasattr(val_results.seg, 'recall'):
            print(f"  Segmentacijos atšaukimas: {val_results.seg.recall:.4f}")

    # Eksportavimo informacija
    print("\nEksportuoti modeliai:")
    if export_success['pt']:
        print(f"  PyTorch (.pt): {pt_save_path}")
    if export_success['onnx']:
        print(f"  ONNX (.onnx): {onnx_save_path}")
    if export_success['onnx_640']:
        print(f"  ONNX 640x640 (.onnx): {onnx_640_save_path}")

    print("\nRezultatai išsaugoti:")
    print(f"  Projekto direktorija: {project_dir}")
    print(f"  Apmokymo grafikai: {os.path.join(project_dir, 'model')}")
    print(f"  Validavimo rezultatai: {os.path.join(results_dir, 'validation')}")
    print("="*50)

    return best_model, results, val_results, best_model_path

# Vykdyti funkciją tik jei turime būtinus importus ir kintamuosius
try:
    import torch
    import os

    # Patikrinti, ar reikalingi kintamieji egzistuoja aplinkoje
    required_vars = ['project_dir', 'exports_dir', 'results_dir', 'dataset_yaml']
    missing_vars = [var for var in required_vars if var not in globals()]

    if missing_vars:
        print(f"Įspėjimas: Trūksta reikalingų kintamųjų: {missing_vars}")
        print("Prašome apibrėžti šiuos kintamuosius prieš paleidžiant šią ląstelę.")
    else:
        print(f"Naudojamas įrenginys: {'CUDA' if torch.cuda.is_available() else 'CPU'}")

        # Apmokyti modelį su svertiniu duomenų krovikliu, integruotu validavimu ir eksportavimu
        trained_model, training_results, validation_results, model_path = train_and_export_model()
        print("Apmokymas, validavimas ir eksportavimas sėkmingai baigti!")
except ImportError as e:
    print(f"Trūksta būtinų importų: {str(e)}")
    print("Prašome pirma įdiegti reikalingus paketus.")
except Exception as e:
    print(f"Klaida bendrame vykdyme: {str(e)}")
    print("Kai kurie žingsniai galbūt buvo sėkmingai baigti. Patikrinkite išvesties direktoriją dėl rezultatų.")

Naudojamas įrenginys: CUDA


100%|██████████| 6.74M/6.74M [00:00<00:00, 212MB/s]


Pradedamas apmokymas su paviršiaus tipo svoriais...
Tai užtruks nemažai laiko.
WARNING ⚠️ 'name=model' automatically updated to 'name=yolov8n-seg'.
Ultralytics 8.3.126 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)


engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=disk, cfg=None, classes=None, close_mosaic=15, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/drive/MyDrive/Dangos Nustatymas/road_seg_20250504_132128/dataset.yaml, degrees=20.0, deterministic=True, device=cuda:0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=130, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.15, hsv_s=0.5, hsv_v=0.5, imgsz=1024, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0005, lrf=0.001, mask_ratio=4, max_det=300, mixup=0.15, mode=train, model=yolov8n-seg.pt, momentum=0.937, mosaic=0.6, multi_scale=False, name=yolov8n-seg, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=20, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile=Fals

100%|██████████| 755k/755k [00:00<00:00, 125MB/s]

...

WARNING ⚠️ Model does not support 'augment=True', reverting to single-scale prediction.
WARNING ⚠️ Model does not support 'augment=True', reverting to single-scale prediction.


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):  97%|█████████▋| 33/34 [00:03<00:00,  6.58it/s]

WARNING ⚠️ Model does not support 'augment=True', reverting to single-scale prediction.


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 34/34 [00:03<00:00,  9.54it/s]


                   all        267        354      0.947      0.833      0.881      0.723      0.943      0.828      0.878      0.731
Speed: 0.3ms preprocess, 2.1ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to /content/drive/MyDrive/Dangos Nustatymas/road_seg_20250504_132128/yolov8n-seg
Apmokymas baigtas!
Klaida bendrame vykdyme: 'function' object has no attribute 'replace'
Kai kurie žingsniai galbūt buvo sėkmingai baigti. Patikrinkite išvesties direktoriją dėl rezultatų.


In [ ]:
#@title 7. YOLOv11 Modelio Eksportavimo Tęsimas ir Užbaigimas

def finish_model_export():
    """Užbaigti modelio eksportavimą ir santrauką po ankstesnio skripto vykdymo"""
    # Importuoti reikalingus paketus
    import os
    import glob
    import torch
    import logging
    import shutil
    import time
    from pathlib import Path

    # Nustatyti bazinį žurnalizavimą
    logging.basicConfig(level=logging.INFO,
                      format='%(asctime)s - %(levelname)s - %(message)s')
    logger = logging.getLogger('export_fix')

    # Patikrinti, ar reikalingi kintamieji egzistuoja globaliai
    global project_dir, exports_dir, results_dir

    # Inicializuoti lokalias kintamųjų versijas
    local_project_dir = None
    local_exports_dir = None
    local_results_dir = None

    # Patikrinti kintamuosius ir prireikus priskirti lokaliai
    try:
        local_project_dir = project_dir
        print(f"Naudojama project_dir iš globalaus konteksto: {local_project_dir}")
    except NameError:
        local_project_dir = input("Įveskite project_dir kelią: ")

    try:
        local_exports_dir = exports_dir
        print(f"Naudojama exports_dir iš globalaus konteksto: {local_exports_dir}")
    except NameError:
        local_exports_dir = input("Įveskite exports_dir kelią: ")

    try:
        local_results_dir = results_dir
        print(f"Naudojama results_dir iš globalaus konteksto: {local_results_dir}")
    except NameError:
        local_results_dir = input("Įveskite results_dir kelią: ")

    logger.info("Pradedamas modelio eksportavimo užbaigimas...")
    print("Pradedamas modelio eksportavimo užbaigimas...")

    # Rasti geriausio modelio kelią
    best_model_path = '/content/drive/MyDrive/Dangos Nustatymas/road_seg_20250504_132128/yolov8n-seg/weights/best.pt'

    if not os.path.exists(best_model_path):
        # Bandyti rasti geriausio modelio failą
        potential_models = glob.glob(os.path.join(local_project_dir, 'model', 'weights', '*.pt'))
        if potential_models:
            best_model_path = potential_models[0]
            logger.info(f"Geriausias modelis nerastas numatytame kelyje. Naudojamas rastas modelis: {best_model_path}")
        else:
            logger.error("Nepavyko rasti apmokyto modelio. Patikrinkite išvesties direktorijas.")
            print("Nepavyko rasti apmokyto modelio. Patikrinkite išvesties direktorijas.")
            return None

    logger.info(f"Naudojamas modelis: {best_model_path}")
    print(f"Naudojamas modelis: {best_model_path}")

    # Įkelti geriausią modelį
    try:
        from ultralytics import YOLO
        best_model = YOLO(best_model_path)

        # Išgauti modelio pavadinimą iš objekto, o ne funkcijos
        if hasattr(best_model, 'type'):
            model_type = best_model.type
            # Patikrinti, ar model_type yra string ir turi 'segment' pradžioje
            if isinstance(model_type, str) and model_type.startswith('segment/'):
                model_name = model_type.replace('segment/', '')
            else:
                # Jei netinkamo formato, ištraukti modelio pavadinimą iš failo kelio
                model_name = Path(best_model_path).stem
                if model_name == 'best' or model_name == 'last':
                    model_name = 'yolov8n-seg'  # Numatytasis jei negalima nustatyti
        else:
            # Jei nėra type atributo, išgauti iš failo kelio
            model_name = Path(best_model_path).stem
            if model_name == 'best' or model_name == 'last':
                model_name = 'yolov8n-seg'  # Numatytasis jei negalima nustatyti

        logger.info(f"Modelio pavadinimas: {model_name}")
        print(f"Modelio pavadinimas: {model_name}")

        # Apibrėžti išvesties failų pavadinimus ir kelius
        pt_filename = f'model_road_seg_{model_name}.pt'
        onnx_filename = f'model_road_seg_{model_name}.onnx'
        onnx_640_filename = f'model_road_seg_{model_name}_640.onnx'

        pt_save_path = os.path.join(local_exports_dir, pt_filename)
        onnx_save_path = os.path.join(local_exports_dir, onnx_filename)
        onnx_640_save_path = os.path.join(local_exports_dir, onnx_640_filename)

        # Užtikrinti, kad eksportų direktorija egzistuoja
        os.makedirs(local_exports_dir, exist_ok=True)

        # Eksportuoti modelius į skirtingus formatus
        logger.info("\nEksportuojami modeliai...")
        print("\nEksportuojami modeliai...")
        export_success = {'pt': False, 'onnx': False, 'onnx_640': False}

        # Eksportuoti į .pt formatą (PyTorch)
        try:
            best_model.save(pt_save_path)  # Naudoti save .pt formatui
            logger.info(f"PyTorch modelis eksportuotas į: {pt_save_path}")
            print(f"PyTorch modelis eksportuotas į: {pt_save_path}")
            export_success['pt'] = True
        except Exception as e:
            logger.error(f"Klaida eksportuojant PyTorch modelį: {str(e)}")
            print(f"Klaida eksportuojant PyTorch modelį: {str(e)}")
            # Bandyti alternatyvią kopijavimo metodiką
            try:
                shutil.copy(best_model_path, pt_save_path)
                logger.info(f"PyTorch modelis nukopijuotas į: {pt_save_path}")
                print(f"PyTorch modelis nukopijuotas į: {pt_save_path}")
                export_success['pt'] = True
            except Exception as e:
                logger.error(f"Klaida kopijuojant PyTorch modelį: {str(e)}")
                print(f"Klaida kopijuojant PyTorch modelį: {str(e)}")

        # Eksportuoti į ONNX formatą
        try:
            logger.info("Eksportuojama į ONNX formatą su išsamiais parametrais...")
            print("Eksportuojama į ONNX formatą su išsamiais parametrais...")

            # Eksportuoti su detaliais parametrais geresniam suderinamumui
            # Use the correct export syntax based on the YOLO version
            try:
                # First try with regular keyword arguments
                best_model.export(format='onnx', imgsz=1024, simplify=True, opset=12,
                                 half=False, int8=False, dynamic=True, verbose=True)

                # If the export completed but didn't save where we want, try to find and move it
                exported_path = os.path.join(os.path.dirname(best_model_path), f"{Path(best_model_path).stem}.onnx")
                if os.path.exists(exported_path) and exported_path != onnx_save_path:
                    shutil.copy(exported_path, onnx_save_path)
                    logger.info(f"ONNX model copied from {exported_path} to {onnx_save_path}")
            except Exception as export_err:
                logger.warning(f"First export attempt failed: {str(export_err)}")
                # Try alternative export methods
                try:
                    # Try the export method as a function with positional arguments
                    export_path = best_model.export(format='onnx')
                    if export_path and os.path.exists(export_path) and export_path != onnx_save_path:
                        shutil.copy(export_path, onnx_save_path)
                        logger.info(f"ONNX model copied from {export_path} to {onnx_save_path}")
                except Exception as alt_err:
                    logger.warning(f"Alternative export attempt failed: {str(alt_err)}")
                    # Try using the command-line interface method
                    try:
                        import subprocess
                        cmd = f"yolo export model={best_model_path} format=onnx imgsz=1024 simplify=True opset=12"
                        subprocess.run(cmd, shell=True, check=True)
                        # Look for the exported model
                        potential_paths = [
                            os.path.join(os.path.dirname(best_model_path), f"{Path(best_model_path).stem}.onnx"),
                            os.path.join(local_project_dir, f"{Path(best_model_path).stem}.onnx"),
                            os.path.join(os.getcwd(), f"{Path(best_model_path).stem}.onnx")
                        ]
                        for path in potential_paths:
                            if os.path.exists(path):
                                shutil.copy(path, onnx_save_path)
                                logger.info(f"ONNX model copied from {path} to {onnx_save_path}")
                                break
                    except Exception as cmd_err:
                        logger.error(f"Command-line export attempt failed: {str(cmd_err)}")
                        print(f"All ONNX export methods failed. Check YOLO version and documentation.")

            if os.path.exists(onnx_save_path):
                logger.info(f"ONNX modelis eksportuotas į: {onnx_save_path}")
                print(f"ONNX modelis eksportuotas į: {onnx_save_path}")
                export_success['onnx'] = True

                # Patikrinti ONNX modelį
                try:
                    import onnx
                    onnx_model = onnx.load(onnx_save_path)
                    onnx.checker.check_model(onnx_model)
                    logger.info("ONNX modelis sėkmingai patikrintas - teisingas formatas")
                    print("ONNX modelis sėkmingai patikrintas - teisingas formatas")
                except Exception as e:
                    logger.warning(f"ONNX modelio validavimo įspėjimas: {str(e)}")
                    print(f"ONNX modelio validavimo įspėjimas: {str(e)}")
            else:
                # Ieškoti eksportuoto ONNX failo kitose vietose
                potential_onnx = []
                for root, dirs, files in os.walk(local_project_dir):
                    for file in files:
                        if file.endswith('.onnx'):
                            potential_onnx.append(os.path.join(root, file))

                if potential_onnx:
                    # Nukopijuoti rastą ONNX failą į reikiamą vietą
                    shutil.copy(potential_onnx[0], onnx_save_path)
                    logger.info(f"ONNX modelis rastas ir nukopijuotas į: {onnx_save_path}")
                    print(f"ONNX modelis rastas ir nukopijuotas į: {onnx_save_path}")
                    export_success['onnx'] = True
                else:
                    logger.error("ONNX eksportavimas baigtas, bet failas nerastas")
                    print("ONNX eksportavimas baigtas, bet failas nerastas")
        except Exception as e:
            logger.error(f"Klaida eksportuojant ONNX modelį: {str(e)}")
            print(f"Klaida eksportuojant ONNX modelį: {str(e)}")

        # Eksportuoti papildomą ONNX su 640x640 vaizdo dydžiu
        try:
            logger.info("Eksportuojama į ONNX formatą su 640x640 vaizdo dydžiu...")
            print("Eksportuojama į ONNX formatą su 640x640 vaizdo dydžiu...")

            # Tiesioginis eksportavimas su 640x640 vaizdo dydžiu
            # Use the correct export syntax based on the YOLO version for 640x640
            try:
                # First try with regular keyword arguments
                best_model.export(format='onnx', imgsz=640, simplify=True, opset=12,
                                 half=False, int8=False, dynamic=True, verbose=True)

                # If the export completed but didn't save where we want, try to find and move it
                exported_path = os.path.join(os.path.dirname(best_model_path), f"{Path(best_model_path).stem}.onnx")
                if os.path.exists(exported_path) and exported_path != onnx_640_save_path:
                    shutil.copy(exported_path, onnx_640_save_path)
                    logger.info(f"ONNX 640x640 model copied from {exported_path} to {onnx_640_save_path}")
            except Exception as export_err:
                logger.warning(f"First 640x640 export attempt failed: {str(export_err)}")
                # Try alternative export methods
                try:
                    # Try the export method as a function with positional arguments
                    export_path = best_model.export(format='onnx', imgsz=640)
                    if export_path and os.path.exists(export_path) and export_path != onnx_640_save_path:
                        shutil.copy(export_path, onnx_640_save_path)
                        logger.info(f"ONNX 640x640 model copied from {export_path} to {onnx_640_save_path}")
                except Exception as alt_err:
                    logger.warning(f"Alternative 640x640 export attempt failed: {str(alt_err)}")
                    # Try using the command-line interface method
                    try:
                        import subprocess
                        cmd = f"yolo export model={best_model_path} format=onnx imgsz=640 simplify=True opset=12"
                        subprocess.run(cmd, shell=True, check=True)
                        # Look for the exported model
                        potential_paths = [
                            os.path.join(os.path.dirname(best_model_path), f"{Path(best_model_path).stem}.onnx"),
                            os.path.join(local_project_dir, f"{Path(best_model_path).stem}.onnx"),
                            os.path.join(os.getcwd(), f"{Path(best_model_path).stem}.onnx")
                        ]
                        for path in potential_paths:
                            if os.path.exists(path):
                                shutil.copy(path, onnx_640_save_path)
                                logger.info(f"ONNX 640x640 model copied from {path} to {onnx_640_save_path}")
                                break
                    except Exception as cmd_err:
                        logger.error(f"Command-line 640x640 export attempt failed: {str(cmd_err)}")
                        print(f"All ONNX 640x640 export methods failed. Check YOLO version and documentation.")

            if os.path.exists(onnx_640_save_path):
                logger.info(f"ONNX 640x640 modelis eksportuotas į: {onnx_640_save_path}")
                print(f"ONNX 640x640 modelis eksportuotas į: {onnx_640_save_path}")
                export_success['onnx_640'] = True

                # Patikrinti 640x640 ONNX modelį
                try:
                    import onnx
                    onnx_model = onnx.load(onnx_640_save_path)
                    onnx.checker.check_model(onnx_model)
                    logger.info("ONNX 640x640 modelis sėkmingai patikrintas - teisingas formatas")
                    print("ONNX 640x640 modelis sėkmingai patikrintas - teisingas formatas")
                except Exception as e:
                    logger.warning(f"ONNX 640x640 modelio validavimo įspėjimas: {str(e)}")
                    print(f"ONNX 640x640 modelio validavimo įspėjimas: {str(e)}")
            else:
                logger.error(f"ONNX 640x640 eksportavimas baigtas, bet failas nerastas")
                print(f"ONNX 640x640 eksportavimas baigtas, bet failas nerastas")
        except Exception as e:
            logger.error(f"ONNX 640x640 eksportavimas nepavyko: {str(e)}")
            print(f"ONNX 640x640 eksportavimas nepavyko: {str(e)}")

        # Gauti validavimo rezultatus iš failų
        validation_results = None
        val_dir = os.path.join(local_results_dir, 'validation')

        # Generuoti apmokymo santrauką
        print("\n" + "="*50)
        print("APMOKYMO IR EKSPORTAVIMO SANTRAUKA")
        print("="*50)

        # Modelio detalės
        print(f"Modelis: {model_name}")
        print(f"Vaizdo dydis: 1024x1024 (apmokymas), 640x640 (papildomas eksportas)")

        # Try to extract some metrics from validation directory if it exists
        if os.path.exists(val_dir):
            import json
            metrics_files = glob.glob(os.path.join(val_dir, '*.json'))
            if metrics_files:
                try:
                    with open(metrics_files[0], 'r') as f:
                        val_data = json.load(f)
                    if 'metrics' in val_data:
                        print("\nValidavimo našumas:")
                        metrics = val_data['metrics']
                        if 'mAP50(B)' in metrics:
                            print(f"  Dėžutės mAP50: {metrics['mAP50(B)']:.4f}")
                        if 'mAP50-95(B)' in metrics:
                            print(f"  Dėžutės mAP50-95: {metrics['mAP50-95(B)']:.4f}")
                        if 'mAP50(M)' in metrics:
                            print(f"  Segmentacijos mAP50: {metrics['mAP50(M)']:.4f}")
                        if 'mAP50-95(M)' in metrics:
                            print(f"  Segmentacijos mAP50-95: {metrics['mAP50-95(M)']:.4f}")
                except Exception as e:
                    print(f"Nepavyko nuskaityti validavimo metrikų: {str(e)}")

        # Eksportavimo informacija
        print("\nEksportuoti modeliai:")
        if export_success['pt']:
            print(f"  PyTorch (.pt): {pt_save_path}")
        if export_success['onnx']:
            print(f"  ONNX (.onnx): {onnx_save_path}")
        if export_success['onnx_640']:
            print(f"  ONNX 640x640 (.onnx): {onnx_640_save_path}")

        print("\nRezultatai išsaugoti:")
        print(f"  Projekto direktorija: {local_project_dir}")
        print(f"  Apmokymo grafikai: {os.path.join(local_project_dir, 'model')}")
        print(f"  Validavimo rezultatai: {os.path.join(local_results_dir, 'validation')}")
        print(f"  Eksportuoti modeliai: {local_exports_dir}")
        print("="*50)

        return export_success

    except Exception as e:
        logger.error(f"Bendra klaida modelio eksportavimo metu: {str(e)}")
        print(f"Bendra klaida modelio eksportavimo metu: {str(e)}")
        import traceback
        traceback.print_exc()
        return None

# Apibūdinkite nustatytus kintamuosius
project_dir = "/content/drive/MyDrive/Dangos Nustatymas/road_seg_20250504_132128"
exports_dir = "/content/drive/MyDrive/Dangos Nustatymas/road_seg_20250504_132128/exports"
results_dir = "/content/drive/MyDrive/Dangos Nustatymas/road_seg_20250504_132128/results"

# Vykdyti funkciją tik jei turime būtinus importus
try:
    import torch
    import os

    print(f"Naudojamas įrenginys: {'CUDA' if torch.cuda.is_available() else 'CPU'}")

    # Patikrinti anksčiau apibrėžtus kintamuosius
    try:
        print(f"Projekto direktorija: {project_dir}")
        print(f"Eksportavimo direktorija: {exports_dir}")
        print(f"Rezultatų direktorija: {results_dir}")
    except NameError:
        print("Kai kurie kintamieji neapibrėžti. Jie bus užklausti funkcijos vykdymo metu.")

    # Užbaigti modelio eksportavimą
    export_results = finish_model_export()

    if export_results:
        print("Modelio eksportavimas sėkmingai užbaigtas!")
    else:
        print("Modelio eksportavimas nebuvo visiškai sėkmingas. Peržiūrėkite klaidas aukščiau.")
except ImportError as e:
    print(f"Trūksta būtinų importų: {str(e)}")
    print("Prašome pirma įdiegti reikalingus paketus.")
except Exception as e:
    print(f"Klaida bendrame vykdyme: {str(e)}")
    import traceback
    traceback.print_exc()

Naudojamas įrenginys: CUDA
Projekto direktorija: /content/drive/MyDrive/Dangos Nustatymas/road_seg_20250504_132128
Eksportavimo direktorija: /content/drive/MyDrive/Dangos Nustatymas/road_seg_20250504_132128/exports
Rezultatų direktorija: /content/drive/MyDrive/Dangos Nustatymas/road_seg_20250504_132128/results
Naudojama project_dir iš globalaus konteksto: /content/drive/MyDrive/Dangos Nustatymas/road_seg_20250504_132128
Naudojama exports_dir iš globalaus konteksto: /content/drive/MyDrive/Dangos Nustatymas/road_seg_20250504_132128/exports
Naudojama results_dir iš globalaus konteksto: /content/drive/MyDrive/Dangos Nustatymas/road_seg_20250504_132128/results
Pradedamas modelio eksportavimo užbaigimas...
Naudojamas modelis: /content/drive/MyDrive/Dangos Nustatymas/road_seg_20250504_132128/yolov8n-seg/weights/best.pt
Modelio pavadinimas: yolov8n-seg

Eksportuojami modeliai...
PyTorch modelis eksportuotas į: /content/drive/MyDrive/Dangos Nustatymas/road_seg_20250504_132128/exports/model_road